In [2]:
pip install chart_studio

     |████████████████████████████████| 71kB 2.3MB/s 


In [3]:
import pandas as pd
import numpy as np
import pandas as pd
import math


In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# load all data
link_cz = 'https://drive.google.com/open?id=1vFnC3QEEKwWW0acUrdIwMOtzoH0F121c'

fluff_cz, id_cz = link_cz.split('=')

downloaded = drive.CreateFile({'id':id_cz}) 
downloaded.GetContentFile('Conzoom_STORKBH_CSV.csv')  
df_cz = pd.read_csv('Conzoom_STORKBH_CSV.csv',header=0 ,sep=";", index_col=False, decimal=",")

In [0]:
df_cz.head()

,FID,Y,X,CellId,KmsCellId,NBefPer,NBefHus,Dagbefolk,DBefStTot,DBefStO15,DBefStU15,DBefBeskT,EAnvfaktor,Ejerfaktor,MxAldfaktr,Bilfaktor,MxUddfaktr,HIndkfaktr,MxPIndfktr,Formfaktor,Formfakto2,Beskfaktor,Civfaktor,Bornfaktor,Alder011,Alder1216,Alder1722,Alder2329,Alder3039,Alder4049,Alder5059,Alder6065,Alder65_,CzG5TypN,CzG5Typ,CzG5GrpN,CzG5Grp,Region,KomKode
0,0,6163050,725450,725461630,100M_61630_7254,3,1,0,0,0,0,0,3,10,4,9,8,9,8,0,1,9,7,8,0.158205,0.075561,0.07438,0.038961,0.094451,0.191263,0.164109,0.07438,0.12869,Elmely,C5,Lever p6 landet,C,1084,155
1,1,6163050,725650,725661630,100M_61630_7256,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,,,,,1084,155
2,2,6163150,724750,724761631,100M_61631_7247,4,1,0,0,0,0,0,3,4,2,8,6,9,9,0,8,9,9,9,0.158205,0.075561,0.07438,0.038961,0.094451,0.191263,0.164109,0.07438,0.12869,Landbofamilier,C2,Lever pa landet,C,1084,155
3,3,6163150,724950,724961631,100M_61631_7249,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,,,,,1084,155
4,4,6163150,725550,725561631,100M_61631_7255,0,0,105,0,0,0,105,3,10,0,0,0,0,0,0,0,9,0,0,0.158205,0.075561,0.07438,0.038961,0.094451,0.191263,0.164109,0.07438,0.12869,Elmely,C5,Lever p6 landet,C,1084,155


In [0]:
data = {'municipality': ['København','Frederiksberg', 'Ballerup', 'Brøndby', 'Dragør', 'Gentofte', 'Gladsaxe', 'Glostrup','Herlev',
            'Albertslund','Hvidovre', 'Høje-Taastrup', 'Rødovre', 'Ishøj','Tårnby', 'Vallensbæk'],
        'KomKode': [101, 147, 151, 153, 155,157,159,161,163,165,167,169,175,183,185,187]}
df_kommoune_koder = pd.DataFrame(data, columns=['KomKode','municipality'])      

In [0]:
focus_municipality = ['København','Frederiksberg', 'Ballerup', 'Brøndby', 'Dragør', 'Gentofte', 'Gladsaxe', 'Glostrup','Herlev',
                      'Albertslund','Hvidovre', 'Høje-Taastrup', 'Rødovre', 'Ishøj','Tårnby', 'Vallensbæk']

In [0]:
df_cz = df_cz[df_cz['KomKode'].isin(df_kommoune_koder['KomKode'].values)]
df_cz = df_cz.merge(df_kommoune_koder, how = 'left')

In [0]:
df_cz.head(1)

,FID,Y,X,CellId,KmsCellId,NBefPer,NBefHus,Dagbefolk,DBefStTot,DBefStO15,DBefStU15,DBefBeskT,EAnvfaktor,Ejerfaktor,MxAldfaktr,Bilfaktor,MxUddfaktr,HIndkfaktr,MxPIndfktr,Formfaktor,Formfakto2,Beskfaktor,Civfaktor,Bornfaktor,Alder011,Alder1216,Alder1722,Alder2329,Alder3039,Alder4049,Alder5059,Alder6065,Alder65_,CzG5TypN,CzG5Typ,CzG5GrpN,CzG5Grp,Region,KomKode,municipality
0,0,6163050,725450,725461630,100M_61630_7254,3,1,0,0,0,0,0,3,10,4,9,8,9,8,0,1,9,7,8,0.158205,0.075561,0.07438,0.038961,0.094451,0.191263,0.164109,0.07438,0.12869,Elmely,C5,Lever p6 landet,C,1084,155,Dragør


In [0]:
'bilfaktor','MxUddfaktr','NbefPer','Dagbefolk','HIndkfaktr'

In [0]:
# load all data
link_feb = 'https://drive.google.com/open?id=1vZDH_QZmct3Z1BbYdIKrq3ZqhvSV4xAh'
link_mar_apr = 'https://drive.google.com/open?id=16prdgB8ud13myHPlczGZpaa-BpUPpf35'

fluff_feb, id_feb = link_feb.split('=')
fluff_mar_apr, id_mar_apr = link_mar_apr.split('=')

downloaded = drive.CreateFile({'id':id_feb}) 
downloaded.GetContentFile('Fremkommelighedsanalyse_med_fraktil_og_potentialeHast_Feb.csv')  
df_feb = pd.read_csv('Fremkommelighedsanalyse_med_fraktil_og_potentialeHast_Feb.csv',sep=";",
                 header= 2,index_col=False,decimal=",")

downloaded = drive.CreateFile({'id':id_mar_apr}) 
downloaded.GetContentFile('Fremkommelighedsanalyse_med_fraktil_og_potentiale_Hast_MarApr.csv')  
df_mar_apr = pd.read_csv('Fremkommelighedsanalyse_med_fraktil_og_potentiale_Hast_MarApr.csv',sep=";",
                 header= 2,index_col=False,decimal=",")

In [0]:
cols = ['StopPointSectionMunicipalityDisplayName','Hastighed_observeret__netto__km_t_','Hastighed_observeret__brutto__km_t_']
df_mar_apr_avg_hast = df_mar_apr[df_mar_apr['StopPointSectionMunicipalityDisplayName'].isin(focus_municipality)][cols]

df_mar_apr_avg_hast = df_mar_apr_avg_hast.groupby('StopPointSectionMunicipalityDisplayName', as_index=False).agg(['sum', 'count'])

df_mar_apr_avg_hast['avg_speed_netto'] = df_mar_apr_avg_hast['Hastighed_observeret__netto__km_t_']['sum'].values/df_mar_apr_avg_hast['Hastighed_observeret__netto__km_t_']['count'].values
df_mar_apr_avg_hast['avg_speed_brutto'] = df_mar_apr_avg_hast['Hastighed_observeret__brutto__km_t_']['sum'].values/df_mar_apr_avg_hast['Hastighed_observeret__brutto__km_t_']['count'].values

df_mar_apr_avg_hast = df_mar_apr_avg_hast[['avg_speed_netto','avg_speed_brutto']]

In [0]:
# extract the data for average speed febuar (prior to covid)
df_feb_avg_hast = df_feb[df_feb['StopPointSectionMunicipalityDisplayName'].isin(focus_municipality)][cols]

df_feb_avg_hast = df_feb_avg_hast.groupby('StopPointSectionMunicipalityDisplayName', as_index=False).agg(['sum', 'count'])

df_feb_avg_hast['avg_speed_netto'] = df_feb_avg_hast['Hastighed_observeret__netto__km_t_']['sum'].values/df_feb_avg_hast['Hastighed_observeret__netto__km_t_']['count'].values
df_feb_avg_hast['avg_speed_brutto'] = df_feb_avg_hast['Hastighed_observeret__brutto__km_t_']['sum'].values/df_feb_avg_hast['Hastighed_observeret__brutto__km_t_']['count'].values

df_feb_avg_hast = df_feb_avg_hast[['avg_speed_netto','avg_speed_brutto']]

In [0]:
# joining avarage speed for febuar and covid periode
df_feb_avg_hast.columns = ['avg_speed_netto_feb', 'avg_speed_brutto_feb']
df_mar_apr_avg_hast.columns = ['avg_speed_netto_mar_apr', 'avg_speed_brutto_mar_apr']
df_avg_hast = df_feb_avg_hast.merge(df_mar_apr_avg_hast, how = 'inner',left_index = True, right_index = True)
df_avg_hast = df_avg_hast[['avg_speed_brutto_feb','avg_speed_brutto_mar_apr']]
df_avg_hast.reset_index(inplace=True)
df_avg_hast['Speed_change'] = (df_avg_hast['avg_speed_brutto_mar_apr'].values/df_avg_hast['avg_speed_brutto_feb'].values -1)
df_hast = df_avg_hast[['StopPointSectionMunicipalityDisplayName','Speed_change']]
df_hast.rename(columns={'StopPointSectionMunicipalityDisplayName': "municipality", 'Speed_change': "Speed_change"},inplace =True)

In [14]:
df_hast

,municipality,Speed_change
0,Albertslund,0.072839
1,Ballerup,0.063975
2,Brøndby,0.059696
3,Dragør,0.044231
4,Frederiksberg,0.064706
5,Gentofte,0.062846
6,Gladsaxe,0.049720
7,Glostrup,0.062729
8,Herlev,0.066371
9,Hvidovre,0.055412


('bilfaktor', 'MxUddfaktr', 'NbefPer', 'Dagbefolk', 'HIndkfaktr')

In [0]:
df_cz_meanb = df_cz[['municipality','NBefPer',	'NBefHus',	'Dagbefolk','DBefBeskT']].groupby('municipality').agg(np.sum)
df_cz_meanf = df_cz[['municipality','EAnvfaktor',	'Ejerfaktor',	'MxAldfaktr',	'Bilfaktor',	'MxUddfaktr',	
                    'HIndkfaktr',	'MxPIndfktr',	'Formfaktor', 'Formfakto2',	'Beskfaktor',	'Civfaktor',	'Bornfaktor']].groupby('municipality').agg(np.sum)

In [0]:
df_cz_mean

,NBefPer,NBefHus,Dagbefolk,DBefStTot,DBefStO15,DBefStU15,DBefBeskT,EAnvfaktor,Ejerfaktor,MxAldfaktr,Bilfaktor,MxUddfaktr,HIndkfaktr,MxPIndfktr,Formfaktor,Formfakto2,Beskfaktor,Civfaktor,Bornfaktor
municipality,,,,,,,,,,,,,,,,,,,
Albertslund,35.018111,15.824062,29.406210,5.799483,1.961190,3.838292,23.606727,4.053040,3.946960,5.239327,4.728331,4.873221,5.371281,5.159120,0.0,4.815006,3.131953,4.852523,5.377749
Ballerup,31.638611,14.599866,28.640615,6.010020,2.708751,3.301269,22.630595,3.758851,3.612558,5.635939,5.828991,5.936540,6.694723,6.632599,0.0,6.452906,4.619238,6.126253,6.156313
Brøndby,36.007368,15.943158,26.887368,6.090526,2.912632,3.177895,20.796842,3.902105,3.635789,5.255789,5.031579,4.694737,5.718947,5.490526,0.0,5.443158,4.266316,5.232632,5.367368
Dragør,23.584041,10.008489,7.950764,3.310696,0.336163,2.974533,4.640068,3.259762,3.220713,6.616299,6.553480,6.789474,7.887946,7.928693,0.0,8.246180,4.870968,6.791171,6.726655
Frederiksberg,142.392012,73.102071,105.886095,52.609467,36.578402,16.031065,53.276627,7.677515,7.097633,4.646450,3.600592,8.310651,6.800296,7.294379,0.0,7.545858,6.102071,4.766272,5.409763
Gentofte,36.599897,16.228557,23.987673,7.855162,2.611710,5.243451,16.132512,4.597329,3.907550,6.430919,6.738059,8.950693,8.708269,8.933231,0.0,9.015922,5.142270,6.551618,7.632768
Gladsaxe,40.886279,18.660692,31.872064,6.920890,2.028430,4.892460,24.951174,4.428925,4.187268,5.273795,5.663782,6.806551,6.872064,6.939431,0.0,6.842398,5.753399,5.977750,6.564277
Glostrup,33.760498,15.979782,32.220840,3.449456,0.320373,3.129082,28.771384,4.141524,3.965785,4.990669,5.269051,5.264386,6.090202,6.040435,0.0,5.958009,5.278383,5.566096,5.670295
Herlev,35.085350,16.357962,22.573248,5.384713,1.789809,3.594904,17.188535,4.092994,4.007643,5.543949,5.482803,5.731210,6.319745,6.301911,0.0,6.170701,4.616561,5.871338,6.142675


In [9]:
#Creating format fitting for bokeh
source = ColumnDataSource(df_cz_meanb)

#Creating a list for the x-axis on the bar-chart. 
x_range = [str(m) for m in source.data['municipality']]

#Generating 14 colors for the different focus crimes  
colors = bokeh.palettes.Category20[20]

#Creating figure frame with title, x -and y labels and disabling toolbar. 
p = figure(x_range=x_range,
           title='Histogram',
           x_axis_label='Municipality',
           y_axis_label='Population',
           plot_width=950,plot_height=500,
           toolbar_location=None)

bar ={} # to store vbars
for indx,i in enumerate(source.column_names[1:]):
    bar[i] = p.vbar(x='municipality', top=i, source= source, width=0.8, 
                 fill_alpha=0.5,line_color=None,
                 color=colors[indx*2],visible=False) 
    
items = [(i,[bar[i]]) for i in source.column_names[1:]] #Creating list of tuples with focus crime name and belonging bar chart. 
legend = Legend(items=items,location=(0,90)) # Creating legends with 'items'
p.add_layout(legend,'left') # Adding legends to ´p´ and setting location.
p.xaxis.major_label_orientation = 1.2
p.legend.click_policy="hide"
p.legend.title = 'To see a periode, choose one below'

show(p)

In [0]:
#Creating format fitting for bokeh
source = ColumnDataSource(df_cz_meanf)

#Creating a list for the x-axis on the bar-chart. 
x_range = [str(m) for m in source.data['municipality']]

#Generating 14 colors for the different focus crimes  
colors = bokeh.palettes.Category20[20]

#Creating figure frame with title, x -and y labels and disabling toolbar. 
p = figure(x_range=x_range,
           title='Histogram',
           x_axis_label='Municipality',
           y_axis_label='Population',
           plot_width=950,plot_height=500,
           toolbar_location=None)

bar ={} # to store vbars
for indx,i in enumerate(source.column_names[1:]):
    bar[i] = p.vbar(x='municipality', top=i, source= source, width=0.8, 
                 fill_alpha=0.5,line_color=None,
                 color=colors[indx],visible=False) 
    
items = [(i,[bar[i]]) for i in source.column_names[1:]] #Creating list of tuples with focus crime name and belonging bar chart. 
legend = Legend(items=items,location=(0,90)) # Creating legends with 'items'
p.add_layout(legend,'left') # Adding legends to ´p´ and setting location.
p.xaxis.major_label_orientation = 1.2
p.legend.click_policy="hide"
p.legend.title = 'To see a periode, choose one below'

show(p)

In [0]:
def scatter_plot(data,title, x_lab,x,y,TOOLTIPS):
  source = ColumnDataSource(data = data)
  f = figure(title = title,
           x_axis_label=x_lab,
           y_axis_label='speed change %',      
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS)
  # center title
  f.title.align = 'center'
  # set title font size
  f.title.text_font_size = '20px'
  # set title font type
  f.title.text_font = 'times'
  # set axes labels font size
  f.axis.axis_label_text_font_size = '16px'
  # set axes labels font type
  f.axis.axis_label_text_font = 'times'

  # Create plot
  f.circle(x = x, y = 'Speed_change', size=10,source=source)
  # show plot
  show(f)

In [0]:
#TOOLTIPS = [("Municipality: ", "@municipality"),
 #           ("Median age", "@procent"),
 #           ("Speed change","@Speed_change")]

 
##df_cz_meanb = df_cz[['municipality','NBefPer',	'NBefHus',	'Dagbefolk',	'DBefStTot',	'DBefStO15',	'DBefStU15',	
   #                 'DBefBeskT']].groupby('municipality').agg(np.mean)
df_cz_mean_NBefPer_speed = df_cz_meanb[['NBefPer']].merge

KeyError: ignored

In [15]:
df_hast.head()

,municipality,Speed_change
0,Albertslund,0.072839
1,Ballerup,0.063975
2,Brøndby,0.059696
3,Dragør,0.044231
4,Frederiksberg,0.064706


In [26]:
df_cz_meanf[['MxUddfaktr','Bilfaktor']]

,MxUddfaktr,Bilfaktor
municipality,,
Albertslund,3767,3655
Ballerup,8887,8726
Brøndby,4460,4780
Dragør,3999,3860
Frederiksberg,5618,2434
Gentofte,17427,13119
Gladsaxe,11013,9164
Glostrup,3385,3388
Herlev,4499,4304


In [18]:
df_cz_meanb.head()

,NBefPer,NBefHus,Dagbefolk
municipality,,,
Albertslund,27069,12232,22731
Ballerup,47363,21856,42875
Brøndby,34207,15146,25543
Dragør,13891,5895,4683
Frederiksberg,96257,49417,71579


In [0]:
t = df_cz_meanf[['MxUddfaktr','Bilfaktor']].merge(df_avg_hast[['StopPointSectionMunicipalityDisplayName','Speed_change']], how='inner',right_on = 'StopPointSectionMunicipalityDisplayName',
                                              left_index=True)[['StopPointSectionMunicipalityDisplayName','MxUddfaktr','Bilfaktor','Speed_change']]


In [29]:
t.head()

,StopPointSectionMunicipalityDisplayName,MxUddfaktr,Bilfaktor,Speed_change
0,Albertslund,3767,3655,0.072839
1,Ballerup,8887,8726,0.063975
2,Brøndby,4460,4780,0.059696
3,Dragør,3999,3860,0.044231
4,Frederiksberg,5618,2434,0.064706


In [34]:
source = ColumnDataSource(data = t[['StopPointSectionMunicipalityDisplayName','MxUddfaktr','Speed_change']])

TOOLTIPS = [("Municipality: ", "@StopPointSectionMunicipalityDisplayName"),
            ("Udd", "@MxUddfaktr"),
            ("Speed change","@Speed_change")]

f = figure(title = 'udd vs change in avg. speed',
           x_axis_label='udd %',
           y_axis_label='change km/h',      
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8,tooltips=TOOLTIPS)
# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'MxUddfaktr', y = 'Speed_change', size=10,source=source)
# show plot
show(f)

In [4]:
df_cz = pd.read_csv('Conzoom_STORKBH_CSV.csv',header=0 ,sep=";", index_col=False, decimal=",")

FileNotFoundError: [Errno 2] File b'Conzoom_STORKBH_CSV.csv' does not exist: b'Conzoom_STORKBH_CSV.csv'